Data provided hold the historical records of Opportunities (a potential deal with client). Opportunity is presented by the following (main) columns (properties): OpportunityId - to identify a separate Opportunity uniquely, CreatedDate - the date of the record creation, StageName - probability and the short stage description for a successful deal closing (100% Opportunity) Amount - (a current corrected version of) sum of the (potential) deal // ??? CloseDate - forecasted deal closing date Probability - the probability of success for this opportunity at its current stage other fields - properties of the Opportunity

You have the history of interactions with clients (Opportunity is a potential deal).
You should forecast the Probability of successful deal (Opportunity) closing.
Speaking more precisely, to forecast most likely next Probability using the known previous history for this Opportunity (potential deal), namely Stage + Probability changes in previous time.
Also you could use available properties of that Opportunity

In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import tensorflow
from tensorflow import keras
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

print(tensorflow.__version__)

2.8.0


In [2]:
df = pd.read_csv("test_data_full.csv")

In [3]:
missing_values_count = df.isnull().sum()
missing_values_count

Unnamed: 0                            0
Id                                    0
OpportunityId                         0
CreatedById                           0
CreatedDate                           0
CreatedDateForInsert                  0
StageName                             0
Amount                                0
Probability                           0
PrevOpportunityStageUpdate            0
ValidThroughDate                    927
SystemModstamp                        0
IsDeleted                             0
PrevAmount                         3843
PrevCloseDate                      3741
AccountId                             0
RecordTypeId                        216
StageSortOrder                        0
Type                                  0
LeadSource                          681
CampaignId                            0
OwnerId                               0
Territory2Id                          0
FiscalYear                            0
FiscalQuarter                         0


In [4]:
data = df[df['StageName'].str.contains('100%') | df['StageName'].str.contains('0% prob')]

print(data.loc[(data.OpportunityId == "0061300001KubxZAAR")])

data.StageName = data.StageName.apply(lambda x: '1' if '100' in x else '0')



Empty DataFrame
Columns: [Unnamed: 0, Id, OpportunityId, CreatedById, CreatedDate, CreatedDateForInsert, StageName, Amount, Probability, PrevOpportunityStageUpdate, ValidThroughDate, SystemModstamp, IsDeleted, PrevAmount, PrevCloseDate, AccountId, RecordTypeId, StageSortOrder, Type, LeadSource, CampaignId, OwnerId, Territory2Id, FiscalYear, FiscalQuarter, ContactId, PartnerAccountId, Number_of_Seats__c, SCC_Interest__c, Company_s_CRM_Type__c, Deal_type__c, Needs__c, Type_of_Users__c, TAM__c, Secondary_CSM__c, Source_Type__c, Original_Lead_Source__c, Original_Lead_Source_Details__c]
Index: []

[0 rows x 38 columns]


C:\Users\Bohdan Potuzhnyi\AppData\Local\Temp\ipykernel_11368\1553354875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.StageName = data.StageName.apply(lambda x: '1' if '100' in x else '0')


In [5]:
series = data.groupby("OpportunityId").OpportunityId.count()
series.sort_values(inplace=True)
series.head(3000)
#opportunity_all = series["OpportunityId"]

OpportunityId
0061300001KIkjMAAT     1
0061B00001RMeAjQAL     1
0061300001HidygAAB     1
0061300001HjfPrAAJ     1
0061300001FVMHDAA5     1
                      ..
0061B00001WwkEyQAJ    25
0061300001NFy3aAAD    25
0061B00001hqcoVQAQ    26
0061B00001ZGR8PQAX    33
0061B00001fgTQlQAM    47
Name: OpportunityId, Length: 803, dtype: int64

In [6]:
all_opportunities = []
for index, row in data.iterrows():
    #print(row)
    all_opportunities.append(row["OpportunityId"])
all_opportunities = np.unique(all_opportunities)
#print(all_opportunities)

In [7]:
### In this part for each opportunity from series DF i will create Array with all data that we
# get from functions described lower
# Firstly i will create array with all opportunities


#now create DF and convert it to array for each op from all opportunities and then push them all
# to one array
all_opportunities_array = []
final_probability_array = []
for opportunity in all_opportunities:
    opp_id = opportunity
    specific_opportinity = data.loc[(data.OpportunityId == opp_id)]
    #status = (specific_opportinity.loc[specific_opportinity.index[0], 'StageName'] == '1')
    # 1-success
    # 0-failure

    #print(status)
    specific_opportinity = specific_opportinity.sort_values(by="CreatedDateForInsert", ascending=True)
    #specific_opportinity

    #print(specific_opportinity)
    start_date = specific_opportinity.loc[specific_opportinity.index[0], "CreatedDate"]
    start_date = datetime.strptime(start_date, '%m/%d/%Y %H:%M')

    res_arr = []
    arr = []
    i = 0
    for current in specific_opportinity["CreatedDateForInsert"]:
        current_date = datetime.strptime(current, '%Y-%m-%d %H:%M:%S')
        res_arr.append(int((current_date - start_date).total_seconds()))
        if i == 0:
            arr.append(int((current_date - start_date).total_seconds()))
            i+=1
        else:
            arr.append(int((current_date - start_date).total_seconds()) - arr[i-1])
            i+=1

    specific_opportinity = specific_opportinity.loc[:, ["Amount", "Probability"]]
    specific_opportinity.insert(2, "Duration", res_arr)
    specific_opportinity.insert(3, "DeltaDuration", arr)
    final_probability = specific_opportinity.iloc[-1, specific_opportinity.columns.get_loc('Probability')]
    final_probability_array.append(final_probability)
    #print(final_probability)
    opp_array = np.array(specific_opportinity)
    all_opportunities_array.append(opp_array)

###

# opp_id = "006a000000xiGkGAAU"
# specific_opportinity = data.loc[(data.OpportunityId == opp_id)]
# print(specific_opportinity)
# status = (specific_opportinity.loc[specific_opportinity.index[0], 'StageName'] == '1')
# # 1-success
# # 0-failure

# #print(status)
# specific_opportinity = specific_opportinity.sort_values(by="CreatedDateForInsert", ascending=True)
# #specific_opportinity

# start_date = specific_opportinity.loc[specific_opportinity.index[0], "CreatedDate"]
# start_date = datetime.strptime(start_date, '%m/%d/%Y %H:%M')

# res_arr = []
# arr = []
# i = 0
# for current in specific_opportinity["CreatedDateForInsert"]:
#     current_date = datetime.strptime(current, '%Y-%m-%d %H:%M:%S')
#     res_arr.append(int((current_date - start_date).total_seconds()))
#     if i == 0:
#         arr.append(int((current_date - start_date).total_seconds()))
#         i+=1
#     else:
#         arr.append(int((current_date - start_date).total_seconds()) - arr[i-1])
#         i+=1
    
# specific_opportinity = specific_opportinity.loc[:, ["CreatedById", "Amount", "Probability"]]
# specific_opportinity.insert(3, "Duration", res_arr)
# specific_opportinity.insert(4, "DeltaDuration", arr)
# final_probability = specific_opportinity.iloc[-1, specific_opportinity.columns.get_loc('Probability')]
# print(final_probability)
# opp_array = np.array(specific_opportinity)
# print(opp_array)


In [8]:
#print(all_opportunities_array)
#final_probability_array

In [9]:
data_model = tensorflow.keras.Sequential(
    [
        tensorflow.keras.layers.Dense(128),
        tensorflow.keras.layers.Dense(1)
    ]
)

data_model.compile(loss = tensorflow.losses.MeanSquaredError(),
                   optimizer = tensorflow.optimizers.Adam())

data_model.fit(all_opportunities_array, final_probability_array, epochs = 10)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'numpy.int64'>"})

In [ ]:
specific_opportinity.drop_duplicates(subset=['Duration'], inplace=True)
specific_opportinity

In [ ]:
sns.heatmap(specific_opportinity.corr('pearson'), annot=True, fmt='.2f')

In [ ]:
#sns.barplot(x="Duration", y="Probability", data=specific_opportinity)
#sns.scatterplot(x="Duration", y="Probability", data=specific_opportinity)

regressor = RandomForestRegressor(n_estimators=500, random_state=0)

In [ ]:
plt.figure(figsize=(20,7))
X = specific_opportinity['Duration'].values[:,np.newaxis]
# target data is array of shape (n,) 
y = specific_opportinity['Probability'].values

regressor.fit(X, y)

y_prediction_forest = regressor.predict(X)
sns.scatterplot(x=specific_opportinity['Duration'], y = y)
sns.lineplot(x=specific_opportinity['Duration'], y = y_prediction_forest)

print('score -> ', regressor.score(X, y))